In [1]:
import rootutils

rootutils.setup_root("./", indicator=".project-root", pythonpath=True)

%load_ext autoreload
%autoreload 2

import torch
import torch_geometric
from topobenchmarkx.data.datasets import CustomDataset
import hydra
from hydra.experimental import initialize, compose

initialize(config_path="../configs", job_name="job")
config = compose(config_name="train.yaml", return_hydra_config=True)

/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
/usr/local/lib/python3.11/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)


In [22]:
import hashlib

context = "string"
sha1 = hashlib.sha1()
sha1.update(str.encode({"context": context, "seed": 0}))
hash_as_hex = sha1.hexdigest()
# convert the hex back to int and restrict it to the relevant int range
seed = int(hash_as_hex, 16) % 4294967295

TypeError: descriptor 'encode' for 'str' objects doesn't apply to a 'dict' object

In [21]:
int(hash_as_hex, 16) % 4294967295

99912828

In [3]:
# Instantiate loading transforms that will be used during loading
load_transforms = hydra.utils.instantiate(config.transforms)

# Datasethydra.utils.instantiate(cfg.dataset, transform_parameters=cfg.transforms)
dataset = hydra.utils.instantiate(
    config.dataset,
).load(transforms=load_transforms)

Transform parameters are the same, using existing data_dir: /TopoBenchmarkX/datasets/graph/cocitation/Cora/Identity/-4302983350053669791


In [5]:
config.transforms

{'_target_': 'topobenchmarkx.transforms.lifting.DataLiftingTransform', 'lifting': 'HypergraphKHopLifting', 'k_value': 1, 'complex_dim': 'None', 'max_triangles': 'None', 'aggregation_method': 'None'}

In [3]:
config.transforms

{'_target_': 'topobenchmarkx.transforms.lifting.DataLiftingTransform', 'lifting': 'Identity'}

ConfigAttributeError: Key 'transforms' is not in struct
    full_key: transforms
    object_type=dict

In [ ]:
load_transforms

DataLiftingTransform()

In [ ]:
loader = hydra.utils.instantiate(config.dataset)

In [ ]:
dataset = hydra.utils.instantiate(config.dataset).load()

In [ ]:
from torch_geometric.data import Data
from topobenchmarkx.data.datasets import CustomDataset

dataset_lst = [Data(x=dataset[i].x, y=dataset[i].y) for i in range(dataset.len())]
dataset = CustomDataset(dataset_lst)
dataloaders = torch_geometric.data.lightning.LightningDataset(dataset, batch_size=32)
tr = dataloaders.train_dataloader()
next(iter(tr))

DataBatch(x=[534, 7], y=[32], batch=[534], ptr=[33])

In [ ]:
dataset = torch_geometric.datasets.TUDataset(root="../data/graph", name="MUTAG")
dataloaders = torch_geometric.data.lightning.LightningDataset(dataset, batch_size=32)
tr = dataloaders.train_dataloader()
next(iter(tr))

DataBatch(edge_index=[2, 1310], x=[588, 7], edge_attr=[1310, 4], y=[32], batch=[588], ptr=[33])

In [ ]:
from torch_geometric.data import Data


def collate_fn(batch):
    """
    args:
        batch - list of (tensor, label)

    reutrn:
        xs - a tensor of all examples in 'batch' after padding
        ys - a LongTensor of all labels in batch
    """

    values, keys = batch[0], batch[1]
    data = Data()
    for key, value in zip(keys, values):
        data[key] = value

    return data


dataset
a = [collate_fn(dataset[i]) for i in range(dataset.len())]

In [ ]:
torch_geometric.data.Batch(a)

DataBatch(x=[188])

In [ ]:
import numpy as np

np.unique([dict(zip(dataset[i][1], dataset[i][0]))["y"] for i in range(dataset.len())])

array([0, 1])

188

In [ ]:
a_dict = dict(zip(a[1], a[0]))
a_dict.keys()

dict_keys(['x_2', 'num_cells_1', 'laplacian_down_1', 'laplacian_up_2', 'incidence_2', 'x_0', 'num_cells_0', 'laplacian_down_2', 'y', 'laplacian_up_1', 'x_1', 'x', 'incidence_1', 'num_cells_2'])

In [ ]:
a_dict["x"].shape

torch.Size([13, 7])

# Load data

In [ ]:
from topobenchmarkx.data.load.loaders import GraphLoader

data_loader = GraphLoader(config.dataset.parameters)
data = data_loader.load()

TypeError: GraphLoader.__init__() missing 1 required positional argument: 'transform'

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

### Lifting

In [ ]:
# # Load data
# from topobenchmarkx.data.load.loaders import HypergraphLoader

# data_loader = HypergraphLoader(config)
# data = data_loader.load()

In [ ]:
config.keys()

dict_keys(['hydra', 'task_name', 'tags', 'train', 'test', 'ckpt_path', 'seed', 'dataset', 'transform', 'model', 'evaluator', 'callbacks', 'trainer', 'paths', 'extras'])

In [ ]:
config["transform"]

{'_target_': 'topobenchmarkx.data.transforms.lifting.DataLiftingTransform', 'lifting': 'CellCyclesLifting', 'k_value': 1, 'complex_dim': 2, 'max_triangles': 6, 'aggregation_method': 'sum'}

In [ ]:
lifting = hydra.utils.instantiate(config.transform)

In [ ]:
lifting

DataLiftingTransform()

In [ ]:
data = torch_geometric.data.Data()
data.x = torch.zeros([6, 1])
data.edge_index = torch.tensor([[0, 0, 0, 1, 1, 1, 2, 4], [1, 2, 3, 2, 3, 4, 3, 5]])

In [ ]:
lifted_data = lifting(data)

In [ ]:
lifted_data

Data(x=[6, 1], incidence_1=[6, 6], num_hyperedges=6, x_0=[6, 1], x_hyperedges=[6, 1])

In [ ]:
cora = torch_geometric.datasets.Planetoid(
    root="../datasets/graph/", name="Cora", pre_transform=lifting
)

/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/venv_topox/lib/python3.11/site-packages/torch_geometric/data/dataset.py:234: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, pass `force_reload=True` explicitly to reload the dataset.
  warnings.warn(


In [ ]:
cora.data

/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/venv_topox/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:293: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[2708, 1433], num_cells_0=2708, num_cells_1=10556, num_cells_2=2648, incidence_1=[2708, 10556], incidence_2=[10556, 2648], laplacian_up_1=[10556, 10556], laplacian_up_2=[2648, 2648], laplacian_down_2=[10556, 10556], laplacian_down_1=[2708, 2708], x_0=[2708, 1433], x_1=[10556, 1433], x_2=[2648, 1433])

In [ ]:
from topobenchmarkx.data.datasets import CustomDataset

In [ ]:
from torch.utils.data import DataLoader
from torch_geometric.data import Data


def collate_fn(batch):
    """
    args:
        batch - list of (tensor, label)

    reutrn:
        xs - a tensor of all examples in 'batch' after padding
        ys - a LongTensor of all labels in batch
    """

    for b in batch:
        values, keys = b[0], b[1]
        data = Data()
        for key, value in zip(keys, values):
            data[key] = value

    return data


d = DataLoader(dataset=CustomDataset([cora.data]), batch_size=1, collate_fn=collate_fn)
next(iter(d))

Data(num_hyperedges=2708, x_0=[2708, 1433], incidence_1=[2708, 2708], x=[2708, 1433], x_hyperedges=[2708, 1433])

In [ ]:
databatch = torch_geometric.data.Batch.from_data_list(data_lst)

NameError: name 'data_lst' is not defined

In [ ]:
databatch.edge_index[:, :4905]

tensor([[   0,    1,    2,  ..., 2706, 2706, 2707],
        [ 978,  736,  399,  ...,  362,  419,  921]])

In [ ]:
databatch.edge_index[:, 4905:]

tensor([[2708, 2709, 2710,  ..., 5414, 5414, 5415],
        [3686, 3444, 3107,  ..., 3070, 3127, 3629]])

In [ ]:
databatch.edge_index = databatch.edge_index.to_sparse()
databatch.edge_index

AllSetTransformer(
  (layers): ModuleList(
    (0): AllSetTransformerLayer(
      (vertex2edge): AllSetTransformerBlock(
        (multihead_att): MultiHeadAttention()
        (mlp): MLP(
          (0): Linear(in_features=64, out_features=64, bias=False)
          (1): Dropout(p=0.2, inplace=False)
        )
        (ln0): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (edge2vertex): AllSetTransformerBlock(
        (multihead_att): MultiHeadAttention()
        (mlp): MLP(
          (0): Linear(in_features=64, out_features=64, bias=False)
          (1): Dropout(p=0.2, inplace=False)
        )
        (ln0): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader


class TextDataset(Dataset):
    def __init__(self, text_list, labels):
        self.text_list = text_list
        self.labels = labels

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, idx):
        text = self.text_list[idx]
        label = self.labels[idx]

        # You can perform any text preprocessing here if needed
        # For example, tokenization, numerical encoding, etc.

        return {"text": text, "label": label}


# Example usage
text_data = [
    "This is an example.",
    "Another text sample.",
    "PyTorch DataLoader with text.",
]
labels = [0, 1, 0]  # Example labels

# Create an instance of your custom dataset
custom_dataset = TextDataset(text_data, labels)

# Use DataLoader to load batches of data
batch_size = 2
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterate through batches
for batch in data_loader:
    texts = batch["text"]
    labels = batch["label"]

    # Perform your training/validation/test operations here
    print("Texts:", texts)
    print("Labels:", labels)

In [ ]:
data_lst[0].keys()

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
next(iter(dataloader))

In [ ]:
from torch_geometric.loader import DataLoader

dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

next(iter(dataloader))

In [ ]:
config.data

### Loss

In [ ]:
loss = hydra.utils.instantiate(config.model.loss)

### Backbone

In [ ]:
hydra.utils.instantiate(config.model.backbone)

### ReadOut

In [ ]:
readout = hydra.utils.instantiate(config.model.readout)

In [ ]:
readout.parameters()

<generator object Module.parameters at 0x19ca5db60>

### Evaluator

In [ ]:
evaluator = hydra.utils.instantiate(config.evaluator)

In [ ]:
import torch

d = {
    "labels": torch.tensor([0, 1, 2, 1, 2, 2]),
    "logits": torch.tensor(
        [[1, 11, 11], [5, 11, 3], [2, 3, 4], [5, 16, 7], [8, 9, 10], [11, 12, 13]]
    ),
}


evaluator.eval(d)

/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/venv_topox/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'labels': tensor([0, 1, 2, 1, 2, 2]),
 'logits': tensor([[ 1, 11, 11],
         [ 5, 11,  3],
         [ 2,  3,  4],
         [ 5, 16,  7],
         [ 8,  9, 10],
         [11, 12, 13]]),
 'metrics': {'acc': 0.8333333333333334,
  'pre_micro': 0.8333333333333334,
  'pre_macro': 0.5555555555555555,
  'rec_micro': 0.8333333333333334,
  'rec_macro': 0.6666666666666666,
  'f1_micro': 0.8333333333333334,
  'f1_macro': 0.6}}

In [ ]:
d["logits"].argmax(dim=1)

tensor([1, 1, 2, 1, 2, 2])

### Dataloader

In [ ]:
# Load data
from topobenchmarkx.data.load.loaders import HypergraphLoader
from topobenchmarkx.data.dataloader_fullbatch import FullBatchDataModule

data_loader = HypergraphLoader(config)
data = data_loader.load()
dataloader = FullBatchDataModule(data=data)

Loading hypergraph dataset name: cora
number of nodes:2708, feature dimension: 1433
number of hyperedges: 1072
Final num_hyperedges 1392
Final num_nodes 2708
Final num_class 7


FileNotFoundError: [Errno 2] No such file or directory: '/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/data//data_splits/cora/train_prop=0.5/split_0.npz'

In [ ]:
batch = next(iter(dataloader.train_dataloader()))

In [ ]:
batch.x[batch.train_mask]

In [ ]:
a = hydra.utils.instantiate(config.model)

In [ ]:
a.hparams.backbone is a.backbone

In [ ]:
id(a.hparams.backbone) == id(a.backbone)

In [ ]:
b = a.backbone.__class__

In [ ]:
b in []topomodelx.nn.hypergraph.unigcnii.UniGCNII

In [ ]:
import topomodelx